## Introduction

Iceland’s glaciers are shrinking, with seasonal melt–freeze cycles superimposed on a persistent long‑term decline (see: [Icelandic glacier loss - AntarcticGlaciers.org](https://www.antarcticglaciers.org/glaciers-and-climate/glacier-recession/icelandic-glacier-loss/) and [A Revised Snow Cover Algorithm to Improve Discrimination between Snow and Clouds: A Case Study in Gran Paradiso National Park](https://www.mdpi.com/2072-4292/13/10/1957)). To monitor these changes at scale, we need methods that minimize data movement while preserving scientific fidelity. This project leverages the new EOPF Zarr format to exploit chunked, cloud‑native reads for efficient snow and ice mapping from Sentinel‑2.

## Seeds

We predefined seeds for our algorithms using [Icelands land cover data](https://www.natt.is/en/resources/open-data/data-download) to locate glacial areas. For each region of connected glaciers or glaciers located close to each other (= within 5km distance), we set one point in the center of that area. This yields a dataset of 21 points.

In [ ]:
import folium
import geopandas as gpd
seeds_file = "./data/Iceland_Seeds.geojson"
df_seeds = gpd.read_file(seeds_file)
if df_seeds.crs is None:
    df_seeds = df_seeds.set_crs("EPSG:4326")
else:
    df_seeds = df_seeds.to_crs("EPSG:4326")

m = folium.Map(zoom_start=10)
folium.GeoJson(df_seeds).add_to(m)
minx, miny, maxx, maxy = df_seeds.total_bounds
bounds = [[miny, minx], [maxy, maxx]]
m.fit_bounds(bounds)
m

: 

## Zarr Optimize Algorithm

## Sentinel Native Algorithm

## Algorithm Comparison

### Benchmark Zarr Optimize Algorithm

### Benchmark Sentinel Native Algorithm

### Conclusion

## Temporal Glacier Analysis